<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/202210_Bigdata/RDD_%EC%8B%A4%EC%8A%B5_2_%EB%82%98%EC%9D%B4%EB%A5%BC_%EA%B8%B0%EC%A4%80%EC%9C%BC%EB%A1%9C_%EC%B9%9C%EA%B5%AC_%EC%88%98_%EC%84%B8%EC%96%B4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("FriendsByAge")
sc = SparkContext(conf = conf)

In [ ]:
directory = "C:\\Users\\mhso_lec\\study_notebook\\data"
filename  = "fakefriends.csv"

In [ ]:
lines = sc.textFile(f"file:///{directory}\\{filename}")
lines.collect()[:6]

['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21',
 '5,Weyoun,59,318']

# 라인 파싱 함수 생성
- 각 라인을 쪼개고 파싱하는 함수를 만들어 둡니다.

In [ ]:
def parseLine(line):
    fields = line.split(',')
    age = int(fields[2])
    numFriends = int(fields[3])
    
    return age, numFriends

In [ ]:
# 직접 만든 함수를 map에서 실행할 수 있도록 콜백으로 전달합니다.
rdd = lines.map(parseLine)
rdd.collect()[:5]

[(33, 385), (26, 2), (55, 221), (40, 465), (68, 21)]

# Key-Value RDD 생성
- 친구의 `age`를 `key`로 놓고, 개수를 세기 위해 `value`에 1을 갖는 `Key-Value RDD`를 생성합니다.
```
(나이, (친구수, 1))
```

In [ ]:
age_count_rdd = rdd.mapValues(lambda x : (x, 1))
age_count_rdd.collect()[:5]

[(33, (385, 1)), (26, (2, 1)), (55, (221, 1)), (40, (465, 1)), (68, (21, 1))]

`reduceByKey`를 이용해 같은 나이(`key`)를 갖는 데이터를 집계합니다.

In [ ]:
# x와 y는 각 항목을 의미합니다. x는 친구의 수, y는 1
totalsByAge = age_count_rdd.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1] ))
totalsByAge.collect()[:5]

[(33, (3904, 12)),
 (26, (4115, 17)),
 (55, (3842, 13)),
 (40, (4264, 17)),
 (68, (2696, 10))]

In [ ]:
# 평균 친구수 구하기
averageByAge = totalsByAge.mapValues(lambda x : x[0] / x[1])
averageByAge.collect()[:5]

[(33, 325.3333333333333),
 (26, 242.05882352941177),
 (55, 295.53846153846155),
 (40, 250.8235294117647),
 (68, 269.6)]

In [ ]:
results = averageByAge.collect()
for result in results:
    print(result)

(33, 325.3333333333333)
(26, 242.05882352941177)
(55, 295.53846153846155)
(40, 250.8235294117647)
(68, 269.6)
(59, 220.0)
(37, 249.33333333333334)
(54, 278.0769230769231)
(38, 193.53333333333333)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(56, 306.6666666666667)
(43, 230.57142857142858)
(36, 246.6)
(22, 206.42857142857142)
(35, 211.625)
(45, 309.53846153846155)
(60, 202.71428571428572)
(67, 214.625)
(19, 213.27272727272728)
(30, 235.8181818181818)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(42, 303.5)
(49, 184.66666666666666)
(48, 281.4)
(50, 254.6)
(39, 169.28571428571428)
(32, 207.9090909090909)
(58, 116.54545454545455)
(64, 281.3333333333333)
(31, 267.25)
(52, 340.6363636363636)
(24, 233.8)
(20, 165.0)
(62, 220.76923076923077)
(41, 268.55555555555554)
(44, 282.1666666666667)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(28, 209.1)
(66, 276.44444444444446)
(46, 223.69230769230768)
(29, 215.91666666666666)
(18, 343.375)
(47, 233.22222222222

In [ ]:
sc.stop()